Groups based on career length

 - Group **A**: less then 1 year
 - Group **B**: 1 - 2 years
 - Group **C**: 3 - 4 years 
 - Group **D**: 5 - 10 years 
 - Group **E**:  more then 10 years

Groups based on working hours

 - Group **1**: 1 - 2 hours 
 - Group **2**: 3 - 4 hours
 - Group **3**: 5 - 6 hours
 - Group **4**: 7 - 8 hours
 - Group **5**: more then 8 hours

Check for grouping this [link](https://realpython.com/pandas-groupby/)

In [134]:
import pandas as pd
from scipy.stats import chi2_contingency
from numpy import square, divide


In [135]:
data = pd.read_csv("task-3-data-new.csv")
N = len(data.index)
groups = data.groupby(["carrer_length", "working_hours"]).size()
groups_df = groups.unstack(level=1, fill_value=0)
print(N)

146


In [136]:
def count_values(df) -> pd.DataFrame:
    r = []
    cols = df.columns.to_list()
    if "sum_of_rows" in cols:
        cols.remove("sum_of_rows")
    for row in df["sum_of_rows"].drop("sum_of_columns"):
        c = []
        for col in df.loc["sum_of_columns"].drop("sum_of_rows"):
            c.append((row * col) / N)
        r.append(c)
    indexes = df.index.drop("sum_of_columns")
    columns = {i: cols[i] for i in range(0, len(cols))}

    return pd.DataFrame(r, index=indexes).rename(columns=columns)

In [137]:
df = groups.unstack(level=1, fill_value=0)
df_sum = df
df_sum["sum_of_rows"] = df.sum(axis=1).to_list()
df_sum = df_sum.append(df.sum(axis=0).rename("sum_of_columns"))
indexes = ['< 1 year', '1 - 2 years', '3 - 4 years', '5 - 10 years', '> 10 years', 'sum_of_columns']
df_sum = df_sum.reindex(indexes)
df_sum

working_hours,1 - 2 hours,3 - 4 hours,5 - 6 hours,7 - 8 hours,> 8 hours,sum_of_rows
carrer_length,,,,,,
< 1 year,1,1,4,3,0,9
1 - 2 years,2,2,1,2,1,8
3 - 4 years,8,10,13,7,9,47
5 - 10 years,3,9,15,17,7,51
> 10 years,0,1,6,11,13,31
sum_of_columns,14,23,39,40,30,146


In [138]:
df_theor = count_values(df_sum)
with open("tables/summary.tex", "w") as f:
    f.write(df_sum.to_latex())
    f.write(df_theor.to_latex())
df_theor

,1 - 2 hours,3 - 4 hours,5 - 6 hours,7 - 8 hours,> 8 hours
carrer_length,,,,,
< 1 year,0.863014,1.417808,2.404110,2.465753,1.849315
1 - 2 years,0.767123,1.260274,2.136986,2.191781,1.643836
3 - 4 years,4.506849,7.404110,12.554795,12.876712,9.657534
5 - 10 years,4.890411,8.034247,13.623288,13.972603,10.479452
> 10 years,2.972603,4.883562,8.280822,8.493151,6.369863


In [139]:
df_reduced = df.copy()

df_reduced = df_reduced.drop(['1 - 2 years', "< 1 year"])
df_reduced = df_reduced.append(df.loc[['1 - 2 years', "< 1 year"]].sum().rename("< 2 years"))

df_reduced_copy = df_reduced.copy()
df_reduced = df_reduced.append(df_reduced.sum(axis=0).rename("sum_of_columns"))
df_reduced = df_reduced.reindex(["< 2 years", "3 - 4 years", "5 - 10 years", "> 10 years", "sum_of_columns"])
df_theor_reduced = count_values(df_reduced)

with open("tables/reduced-1.tex", "w") as f:
    f.write(df_reduced.to_latex())
    f.write(df_theor_reduced.to_latex())
df_theor_reduced

,1 - 2 hours,3 - 4 hours,5 - 6 hours,7 - 8 hours,> 8 hours
carrer_length,,,,,
< 2 years,1.630137,2.678082,4.541096,4.657534,3.493151
3 - 4 years,4.506849,7.404110,12.554795,12.876712,9.657534
5 - 10 years,4.890411,8.034247,13.623288,13.972603,10.479452
> 10 years,2.972603,4.883562,8.280822,8.493151,6.369863


In [140]:
df_reduced_2 = df_reduced.copy()

df_reduced_2["< 4 hours"] = df_reduced_2["1 - 2 hours"] + df_reduced_2["3 - 4 hours"]
df_reduced_2.drop(["1 - 2 hours", "3 - 4 hours"], axis=1, inplace=True)

df_reduced_2 = df_reduced_2.append((df_reduced_2.loc["< 2 years"] + df_reduced_2.loc["3 - 4 years"]).rename("< 4 years"))
df_reduced_2.drop(["< 2 years", "3 - 4 years"], axis=0, inplace=True)
df_reduced_2 = df_reduced_2.reindex(["< 4 years", "5 - 10 years", "> 10 years", "sum_of_columns"])

df_theor_reduced_2 = count_values(df_reduced_2)
with open("tables/reduced-2.tex", "w") as f:
    f.write(df_reduced_2.to_latex())
    f.write(df_theor_reduced_2.to_latex())
df_theor_reduced_2

,5 - 6 hours,7 - 8 hours,> 8 hours,< 4 hours
carrer_length,,,,
< 4 years,17.095890,17.534247,13.150685,16.219178
5 - 10 years,13.623288,13.972603,10.479452,12.924658
> 10 years,8.280822,8.493151,6.369863,7.856164


In [145]:
df_reduced_copy = df_reduced_2.drop("sum_of_columns").drop("sum_of_rows", axis=1)

part_teor_vals = pd.DataFrame()
for col in df_theor_reduced_2.columns.to_list():
    part_teor_vals[col] = divide(square(df_reduced_copy[col] - df_theor_reduced_2[col]), df_theor_reduced_2[col])
with open("tables/part-teor-vals.tex", "w") as f:
    f.write(part_teor_vals.to_latex())

theor_value = part_teor_vals.sum().sum()
dot = (len(part_teor_vals.index) - 1)*(len(part_teor_vals.columns) - 1)
chi2_table_value = 12.592

if theor_value < chi2_table_value:
    print(f"Not rejecting H0, theoretical value: {theor_value}")
else:
    print(f"Rejecting H0, theoretical value: {theor_value}")

Rejecting H0, theoretical value: 22.551225232676842
